In [54]:
import requests
import pandas as pd
import json

In [55]:
def runGQLquery(query_str,query_key):

    query = { 'nrql': query_str }
    headers = {'Content-Type': 'application/json','API-Key': query_key}
    #headers = {'Content-Type': 'application/json'}
    retry = 3
    url = ('https://api.newrelic.com/graphql')
    success = False
    r_json = {}
    count = 0
    while not success and count < retry:
        r = requests.get(url, data=query_str, headers=headers )
        r_json = r.json()
        success = r.status_code == requests.codes.ok
        count = count + 1

    if not success:
        print('Error getting GraphQl query: {}'.format(r_json))
        return None
    
    return r_json

In [56]:
AllRPMs = '[Your Account ID Here],[Your other Account IDs Here separated by comma]'
#For example:AllRPMs = '2997027,3147675'
AllAPIKeys = '[Your API Key for account 1 Here],[Your API Key for other accounts Here separated by comma]'
#For Example:AllAPIKeys = 'NRAK-XYXYUXYYXXYYXYXYXYXYXYXYXYXYXY,NRAK-XYXYXYXYXYXYXYXYXYXYXYXYXYXYXY'

account_ids = AllRPMs.split(',')
query_keys = AllAPIKeys.split(',')

entityNames = '[Your Host or Entity Name Here]'
#For Example:entityNames = 'i-0dcae3384c4a6fabc'

metricsList = pd.DataFrame(columns=['metricName','entity.name','accountId'])

 
    
for account_id, query_key in zip(account_ids,query_keys):
    query_metriclist = '{\
        actor {\
        account(id: '+account_id+') {\
            id\
            nrql(query: "select uniques(metricName) from Metric since 1 hour ago where entity.name = \''+entityNames+'\' limit max") {\
            results\
            }\
        }\
        }\
    }'
    results = runGQLquery(query_metriclist,query_key)
    #print(results)
    
    prev_metricsList=pd.json_normalize(results['data']['actor']['account']['nrql']['results'])
    allmetrics = prev_metricsList['uniques.metricName'].values[0]
    
    for metric in allmetrics:
        metricsList = metricsList.append({'accountId':account_id,'entity.name':entityNames,'metricName':metric},ignore_index=True)

    
#prev_slo    

#df
metricsList

,metricName,entity.name,accountId
0,host.process.ioWriteBytesPerSecond,i-0dcae3384c4a6fbe8,2997027
1,host.cpuSystemPercent,i-0dcae3384c4a6fbe8,2997027
2,aws.ec2.EBSByteBalance%,i-0dcae3384c4a6fbe8,2997027
3,host.disk.freeBytes,i-0dcae3384c4a6fbe8,2997027
4,host.disk.usedPercent,i-0dcae3384c4a6fbe8,2997027
...,...,...,...
89,host.net.transmitBytesPerSecond,i-0dcae3384c4a6fbe8,2997027
90,host.disk.currentQueueLen,i-0dcae3384c4a6fbe8,2997027
91,host.cpuIdlePercent,i-0dcae3384c4a6fbe8,2997027
92,host.process.cpuPercent,i-0dcae3384c4a6fbe8,2997027


In [57]:
metricsList.to_csv('MetricsOut-May2021.csv', index=False)  